In [1]:
import numpy as np 
import pandas as pd 
np.random.seed(42)  
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns

color = sns.color_palette()
sns.set_style('darkgrid')

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('./processed/merged_data_Y2.csv')
df_y3 = pd.read_csv('./processed/merged_data_Y3.csv')

In [3]:
df_y3

,MemberID,AgeAtFirstClaim,Male,Female,Unknown,ClaimsCount,Unique_ProviderID,Unique_Vendor,Unique_PCP,PayDelay,...,3- 4 months,4- 5 months,5- 6 months,6- 7 months,7- 8 months,8- 9 months,9-10 months,DrugCount,LabCount,DaysInHospital
0,210,35.0,0.0,0.0,1.0,4.0,2.0,2.0,1.0,143.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,1.0,0
1,3197,5.0,0.0,1.0,0.0,11.0,3.0,1.0,1.0,379.0,...,0.0,0.0,1.0,0.0,2.0,0.0,0.0,6.0,0.0,0
2,5187,5.0,1.0,0.0,0.0,13.0,4.0,3.0,1.0,563.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0
3,14033,15.0,0.0,1.0,0.0,8.0,7.0,7.0,1.0,330.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2
4,14661,5.0,0.0,0.0,1.0,3.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49677,99982956,15.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,62.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
49678,99985795,55.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,35.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0
49679,99985996,0.0,0.0,0.0,0.0,18.0,6.0,5.0,1.0,561.0,...,0.0,0.0,1.0,2.0,2.0,3.0,0.0,35.0,0.0,0
49680,99987030,25.0,1.0,0.0,0.0,8.0,3.0,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0


In [5]:
# from itertools import product

# def manual_tune_xgb(X_train, y_train):
#     xgb_params = {
#         'learning_rate': [0.01, 0.1, 1],
#         'max_depth': [3, 5, 7],
#         'subsample': [0.3, 0.5, 0.7],
#         'n_estimators': [100, 300, 500],
#     }

#     best_score = float('inf')
#     best_params = None

#     for lr, depth, subsample, n_est in product(xgb_params['learning_rate'], 
#                                                xgb_params['max_depth'], 
#                                                xgb_params['subsample'], 
#                                                xgb_params['n_estimators']):
#         xgb_model = XGBRegressor(objective='reg:squarederror',
#                                  learning_rate=lr, 
#                                  max_depth=depth, 
#                                  subsample=subsample, 
#                                  n_estimators=n_est)
#         xgb_model.fit(X_train, y_train)
        
#         # 假设使用rmsle作为评估指标
#         predictions = xgb_model.predict(X_train)
#         score = mean_squared_error(y_train, predictions)
        
#         if score < best_score:
#             best_score = score
#             best_params = (lr, depth, subsample, n_est)

#     print("Best Params: ", best_params)
#     print("Best RMSLE: ", best_score)
#     return best_params

# best_params = manual_tune_xgb(X_train, y_train)


In [7]:
# 来自联合特征选择：数值型数据--相关系数法 + 类别型数据--互信息值 40个特征
feature_combination1 = df[['ClaimsCount', 'PayDelay', 'Unique_ProviderID', 'MED', 'EM', 'SCS', 'RAD', 'ANES', 'SDS', 'PL', 'SIS', 'SMS', 'SGS', 'SEOA', 'SNS',
 'SAS', 'SRS', 'SUS', 'SO', 'SMCD', 'Office', 'Independent Lab', 'Outpatient Hospital', 'Inpatient Hospital',
 'Urgent Care', 'Ambulance', 'Home', 'Other_y', 'Surgery', 'Internal', 'Laboratory', 'Pediatrics', 'Rehabilitation','Diagnostic Imaging', 'Anesthesiology', 'Emergency', 
        'General Practice', 'Other_x', 'Obstetrics and Gynecology', 'Pathology']]
# 来自递归特征消除，通过递归地构建线性回归模型来筛选出最有用的特征。33个特征
feature_combination2 = df[['ClaimsCount', 'Diagnostic Imaging', 'Emergency', 'Pathology',
       'Ambulance', 'Home', 'Inpatient Hospital', 'Office', 'Other_y',
       'Outpatient Hospital', 'Urgent Care', 'APPCHOL', 'CANCRM', 'CATAST',
       'LIVERDZ', 'PERINTL', 'PERVALV', 'PNCRDZ', 'RENAL1', 'SEPSIS', 'STROKE',
       '0', '1-2', '3-4', '5+', 'ANES', 'PL', 'SAS', 'SCS', 'SDS', 'SMCD',
       'SMS', 'SO']]
#来自嵌入法使用基于随机森林模型的特征选择方法，自动处理混合特征类型，并给出特征的重要性。27个特征
feature_combination3 = df[['AgeAtFirstClaim', 'ClaimsCount', 'Unique_ProviderID', 'Unique_Vendor', 'PayDelay',
    'Diagnostic Imaging', 'Emergency', 'General Practice', 'Internal', 'Surgery',
    'Ambulance', 'Independent Lab', 'Inpatient Hospital', 'Office', 'Urgent Care',
    'MSC2a3', 'ROAMI', 'TRAUMA', '1-2', 'EM', 'MED', 'PL', 'RAD', 'SDS',
    '0- 1 month', 'DrugCount', 'LabCount']]

In [4]:
feature_combination1

,ClaimsCount,PayDelay,Unique_ProviderID,MED,EM,SCS,RAD,ANES,SDS,PL,...,Laboratory,Pediatrics,Rehabilitation,Diagnostic Imaging,Anesthesiology,Emergency,General Practice,Other_x,Obstetrics and Gynecology,Pathology
0,6.0,308.0,3.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
1,5.0,148.0,4.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,...,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11.0,608.0,8.0,0.0,3.0,1.0,4.0,0.0,1.0,1.0,...,2.0,1.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0
3,12.0,637.0,4.0,3.0,3.0,2.0,1.0,0.0,0.0,3.0,...,2.0,5.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0
4,8.0,320.0,6.0,0.0,6.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51961,16.0,553.0,5.0,3.0,7.0,1.0,0.0,0.0,0.0,5.0,...,6.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0
51962,14.0,1081.0,6.0,2.0,5.0,2.0,1.0,0.0,0.0,3.0,...,1.0,0.0,0.0,1.0,0.0,0.0,8.0,0.0,0.0,0.0
51963,4.0,115.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
51964,6.0,235.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X_train1 = feature_combination1
y_train1 = df['DaysInHospital']  # 目标变量（住院天数）

X_train2 = feature_combination2
y_train2 = df['DaysInHospital']  # 目标变量（住院天数）

X_train3 = feature_combination3
y_train3 = df['DaysInHospital']  # 目标变量（住院天数）

In [9]:
X_test1 = df_y3[['ClaimsCount', 'PayDelay', 'Unique_ProviderID', 'MED', 'EM', 'SCS', 'RAD', 'ANES', 'SDS', 'PL', 'SIS', 'SMS', 'SGS', 'SEOA', 'SNS',
 'SAS', 'SRS', 'SUS', 'SO', 'SMCD', 'Office', 'Independent Lab', 'Outpatient Hospital', 'Inpatient Hospital',
 'Urgent Care', 'Ambulance', 'Home', 'Other_y', 'Surgery', 'Internal', 'Laboratory', 'Pediatrics', 'Rehabilitation','Diagnostic Imaging', 'Anesthesiology', 'Emergency', 
        'General Practice', 'Other_x', 'Obstetrics and Gynecology', 'Pathology']]
y_test = df_y3['DaysInHospital']  # 实际的住院天数，用于评估

X_test2 = df_y3[['ClaimsCount', 'Diagnostic Imaging', 'Emergency', 'Pathology',
       'Ambulance', 'Home', 'Inpatient Hospital', 'Office', 'Other_y',
       'Outpatient Hospital', 'Urgent Care', 'APPCHOL', 'CANCRM', 'CATAST',
       'LIVERDZ', 'PERINTL', 'PERVALV', 'PNCRDZ', 'RENAL1', 'SEPSIS', 'STROKE',
       '0', '1-2', '3-4', '5+', 'ANES', 'PL', 'SAS', 'SCS', 'SDS', 'SMCD',
       'SMS', 'SO']]  # 实际的住院天数，用于评估
y_test = df_y3['DaysInHospital']

X_test3 = df_y3[['AgeAtFirstClaim', 'ClaimsCount', 'Unique_ProviderID', 'Unique_Vendor', 'PayDelay',
    'Diagnostic Imaging', 'Emergency', 'General Practice', 'Internal', 'Surgery',
    'Ambulance', 'Independent Lab', 'Inpatient Hospital', 'Office', 'Urgent Care',
    'MSC2a3', 'ROAMI', 'TRAUMA', '1-2', 'EM', 'MED', 'PL', 'RAD', 'SDS',
    '0- 1 month', 'DrugCount', 'LabCount']]  # 实际的住院天数，用于评估
y_test = df_y3['DaysInHospital']

In [12]:
model = XGBRegressor(objective='reg:squarederror', random_state=42, max_depth=3)
model.fit(X_train1, y_train1)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test1)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Feature combination1:")
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=3)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Feature combination2:")
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=3)
model.fit(X_train3, y_train3)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test3)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Feature combination3:")
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

Feature combination1:
Mean Squared Error (MSE): 0.646256775799199
R-squared: 0.6770075664276588
Feature combination2:
Mean Squared Error (MSE): 0.6454602657525513
R-squared: 0.677405653887575
Feature combination3:
Mean Squared Error (MSE): 0.6756240973715153
R-squared: 0.66233008370353


### max_depth 

In [15]:
model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=2)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')


model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=3)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=4)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=5)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=6)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=7)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')


model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=8)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=9)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=10)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

Mean Squared Error (MSE): 0.6570902801744934
R-squared: 0.6715930933059406
Mean Squared Error (MSE): 0.6454602657525513
R-squared: 0.677405653887575
Mean Squared Error (MSE): 0.6497065141915417
R-squared: 0.6752834229598356
Mean Squared Error (MSE): 0.6571138015014533
R-squared: 0.6715813375906893
Mean Squared Error (MSE): 0.6803883038051081
R-squared: 0.659948982742346
Mean Squared Error (MSE): 0.7243535166095867
R-squared: 0.6379756253896385
Mean Squared Error (MSE): 0.7564328144576934
R-squared: 0.6219427250514742
Mean Squared Error (MSE): 0.796518014707727
R-squared: 0.6019085577300071
Mean Squared Error (MSE): 0.79393285082255
R-squared: 0.6032005958265114


### subsample

In [14]:
model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.1,max_depth=3)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.3,max_depth=3)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.5,max_depth=3)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.7,max_depth=3)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.9,max_depth=3)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=1,max_depth=3)
model.fit(X_train2, y_train2)

# 使用模型预测第三年的住院天数
y_pred = model.predict(X_test2)

# 计算均方误差（MSE）,R2来评估模型的性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

Mean Squared Error (MSE): 0.8093876450630393
R-squared: 0.5954764499621648
Mean Squared Error (MSE): 0.6964396319051389
R-squared: 0.6519266954422467
Mean Squared Error (MSE): 0.6674704245153356
R-squared: 0.6664052048271938
Mean Squared Error (MSE): 0.6680623170241516
R-squared: 0.6661093830903938
Mean Squared Error (MSE): 0.6518017759993782
R-squared: 0.6742362328403241
Mean Squared Error (MSE): 0.6454602657525513
R-squared: 0.677405653887575


In [11]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 之前定义的 k-fold 交叉验证函数
def k_fold_cross_validation(X, y, k, model):
    """
    Implement k-fold cross validation.
    Returns the mean and standard deviation of the mean squared error (MSE).
    """
    np.random.seed(42)
    fold_size = len(X) // k
    indices = np.random.permutation(len(X))
    mse_scores = []

    for i in range(k):
        val_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate([indices[:i * fold_size], indices[(i + 1) * fold_size:]])

        X_train, X_val = X.iloc[train_indices], X.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mse_scores.append(mse)

    return np.mean(mse_scores), np.std(mse_scores)

# 嵌套交叉验证函数，调优 max_depth 参数
def nested_cross_validation(X, y, outer_k=3, inner_k=3, param_grid=None):
    """
    Implements nested cross validation for hyperparameter tuning (max_depth) and model evaluation.
    """
    if param_grid is None:
        param_grid = {'max_depth': [3, 5, 7, 9]}

    outer_mse_scores = []
    best_params = None
    best_mse = float('inf')

    np.random.seed(42)
    indices = np.random.permutation(len(X))
    fold_size_outer = len(X) // outer_k

    for i in range(outer_k):
        val_indices_outer = indices[i * fold_size_outer:(i + 1) * fold_size_outer]
        train_indices_outer = np.concatenate([indices[:i * fold_size_outer], indices[(i + 1) * fold_size_outer:]])

        X_train_outer, X_val_outer = X.iloc[train_indices_outer], X.iloc[val_indices_outer]
        y_train_outer, y_val_outer = y.iloc[train_indices_outer], y.iloc[val_indices_outer]

        for max_depth in param_grid['max_depth']:
            model = XGBRegressor(max_depth=max_depth, objective='reg:squarederror', random_state=42)

            # 内层交叉验证，用于调优 max_depth
            mean_mse, _ = k_fold_cross_validation(X_train_outer, y_train_outer, inner_k, model)

            if mean_mse < best_mse:
                best_mse = mean_mse
                best_params = {'max_depth': max_depth}

        # 用最佳 max_depth 在外层训练集上训练最终模型
        final_model = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
        final_model.fit(X_train_outer, y_train_outer)
        y_pred_outer = final_model.predict(X_val_outer)
        mse_outer = mean_squared_error(y_val_outer, y_pred_outer)
        outer_mse_scores.append(mse_outer)

    return np.mean(outer_mse_scores), np.std(outer_mse_scores), best_params

# 运行多次嵌套交叉验证来确定最佳 max_depth
def run_multiple_experiments(X, y, repetitions=20, outer_k=3, inner_k=3, param_grid=None):
    """
    Perform multiple nested cross-validation experiments to return the best hyperparameters and MSE statistics.
    """
    mse_results = []
    best_params_list = []

    for i in range(repetitions):
        print(f"Running repetition {i+1}/{repetitions}")
        mean_mse, std_mse, best_params = nested_cross_validation(X, y, outer_k=outer_k, inner_k=inner_k, param_grid=param_grid)
        mse_results.append(mean_mse)
        best_params_list.append(best_params)

    return np.mean(mse_results), np.std(mse_results), best_params_list

# 调整 max_depth 参数
param_grid = {'max_depth': [3, 5, 7, 9]}
mean_mse, std_mse, best_params_list = run_multiple_experiments(X_train2, y_train2, repetitions=20, param_grid=param_grid)

# 输出结果
print(f"Mean MSE after 20 repetitions: {mean_mse}")
print(f"Standard deviation of MSE after 20 repetitions: {std_mse}")
print(f"Best max_depth for each run: {best_params_list}")


Running repetition 1/20
Running repetition 2/20
Running repetition 3/20
Running repetition 4/20
Running repetition 5/20
Running repetition 6/20
Running repetition 7/20
Running repetition 8/20
Running repetition 9/20
Running repetition 10/20
Running repetition 11/20
Running repetition 12/20
Running repetition 13/20
Running repetition 14/20
Running repetition 15/20
Running repetition 16/20
Running repetition 17/20
Running repetition 18/20
Running repetition 19/20
Running repetition 20/20
Mean MSE after 20 repetitions: 0.7446449710175636
Standard deviation of MSE after 20 repetitions: 2.220446049250313e-16
Best max_depth for each run: [{'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}]

### subsample

In [13]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 使用之前定义的 k-fold 交叉验证函数
def k_fold_cross_validation(X, y, k, model):
    """
    Implement k-fold cross validation.
    Returns the mean and standard deviation of the mean squared error (MSE).
    """
    np.random.seed(42)
    fold_size = len(X) // k
    indices = np.random.permutation(len(X))
    mse_scores = []

    for i in range(k):
        val_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate([indices[:i * fold_size], indices[(i + 1) * fold_size:]])

        X_train, X_val = X.iloc[train_indices], X.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mse_scores.append(mse)

    return np.mean(mse_scores), np.std(mse_scores)

# 嵌套交叉验证函数，调优 subsample 参数
def nested_cross_validation(X, y, outer_k=3, inner_k=3, param_grid=None):
    """
    Implements nested cross validation for hyperparameter tuning (subsample) and model evaluation.
    """
    if param_grid is None:
        param_grid = {'subsample': [0.3, 0.5, 0.7, 1.0]}

    outer_mse_scores = []
    best_params = None
    best_mse = float('inf')

    np.random.seed(42)
    indices = np.random.permutation(len(X))
    fold_size_outer = len(X) // outer_k

    for i in range(outer_k):
        val_indices_outer = indices[i * fold_size_outer:(i + 1) * fold_size_outer]
        train_indices_outer = np.concatenate([indices[:i * fold_size_outer], indices[(i + 1) * fold_size_outer:]])

        X_train_outer, X_val_outer = X.iloc[train_indices_outer], X.iloc[val_indices_outer]
        y_train_outer, y_val_outer = y.iloc[train_indices_outer], y.iloc[val_indices_outer]

        for subsample in param_grid['subsample']:
            model = XGBRegressor(subsample=subsample, objective='reg:squarederror', random_state=42)

            # 内层交叉验证，用于调优 subsample
            mean_mse, _ = k_fold_cross_validation(X_train_outer, y_train_outer, inner_k, model)

            if mean_mse < best_mse:
                best_mse = mean_mse
                best_params = {'subsample': subsample}

        # 用最佳 subsample 在外层训练集上训练最终模型
        final_model = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
        final_model.fit(X_train_outer, y_train_outer)
        y_pred_outer = final_model.predict(X_val_outer)
        mse_outer = mean_squared_error(y_val_outer, y_pred_outer)
        outer_mse_scores.append(mse_outer)

    return np.mean(outer_mse_scores), np.std(outer_mse_scores), best_params

# 运行多次嵌套交叉验证来确定最佳 subsample
def run_multiple_experiments(X, y, repetitions=20, outer_k=3, inner_k=3, param_grid=None):
    """
    Perform multiple nested cross-validation experiments to return the best hyperparameters and MSE statistics.
    """
    mse_results = []
    best_params_list = []

    for i in range(repetitions):
        print(f"Running repetition {i+1}/{repetitions}")
        mean_mse, std_mse, best_params = nested_cross_validation(X, y, outer_k=outer_k, inner_k=inner_k, param_grid=param_grid)
        mse_results.append(mean_mse)
        best_params_list.append(best_params)

    return np.mean(mse_results), np.std(mse_results), best_params_list

# 调整 subsample 参数
param_grid = {'subsample': [0.3, 0.5, 0.7, 1.0]}
mean_mse, std_mse, best_params_list = run_multiple_experiments(X_train2, y_train2, repetitions=20, param_grid=param_grid)

# 输出结果
print(f"Mean MSE after 20 repetitions: {mean_mse}")
print(f"Standard deviation of MSE after 20 repetitions: {std_mse}")
print(f"Best subsample for each run: {best_params_list}")


Running repetition 1/20
Running repetition 2/20
Running repetition 3/20
Running repetition 4/20
Running repetition 5/20
Running repetition 6/20
Running repetition 7/20
Running repetition 8/20
Running repetition 9/20
Running repetition 10/20
Running repetition 11/20
Running repetition 12/20
Running repetition 13/20
Running repetition 14/20
Running repetition 15/20
Running repetition 16/20
Running repetition 17/20
Running repetition 18/20
Running repetition 19/20
Running repetition 20/20
Mean MSE after 20 repetitions: 0.7898648347938104
Standard deviation of MSE after 20 repetitions: 1.1102230246251565e-16
Best subsample for each run: [{'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}